In [1]:
from nlp_ai_utils import *
from chunks_urls import CHUNKS_URLS

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
TF_ENABLE_ONEDNN_OPTS = 0
URLS = CHUNKS_URLS
GLOVE_URL = 'https://f005.backblazeb2.com/file/glove-embeddings-dims/glove.6B.100d.txt'
LIMIT = 60
LARGER_DATASET_PATH = "../larger_dataset"
PREPROCESSED_CHUNKS_PATH = LARGER_DATASET_PATH + "/preprocessed_data_chunks/"
BASE_FILE_NAME = "chunk_"
FILE_FORMAT = ".csv"
ACTUAL_DATA_SHAPE = 6990280
RANDOM_STATE = 42

In [ ]:
if not os.path.exists(LARGER_DATASET_PATH):
    os.mkdir(LARGER_DATASET_PATH)
if not os.path.exists(PREPROCESSED_CHUNKS_PATH):
    os.mkdir(PREPROCESSED_CHUNKS_PATH)
get_chunks(URLS,LIMIT,1,BASE_FILE_NAME,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

In [6]:
#get all names of downloaded files
all_file_names = get_all_file_names(BASE_FILE_NAME,LIMIT)

In [7]:
#read all chunks into a list
list_dfs = read_chunks(all_file_names,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

In [8]:
def group_up_chunks(dfs):
    #adds up all dataframes together
    return pd.concat(dfs)

In [9]:
#concatenate all chunks into a singular df
df = group_up_chunks(list_dfs)

In [10]:
review_data = df[['text', 'stars']].copy()

In [11]:
review_data.reset_index(inplace = True)
review_data.drop(['index'],axis = 1,inplace = True)

In [12]:
review_data.rename(columns = {'text':'full_review_text','stars':'star_rating'}, inplace = True)

In [13]:
review_data.isnull().sum()
review_data.dropna(inplace = True)

,full_review_text,star_rating
0,decide eat aware go take 2 hour begin end try ...,3.0
1,ve take lot spin class year nothing compare cl...,5.0
2,family diner buffet eclectic assortment large ...,3.0
3,wow yummy different delicious favorite lamb cu...,5.0
4,cute interior owner give u tour upcoming patio...,4.0
...,...,...
6990275,late addition service iccu apple pay iccu debi...,5.0
6990276,spot offer great affordable east weekend paddl...,5.0
6990277,home depot need get lot demential lumber seem ...,4.0
6990278,m feel like ignore caloriecounting indulge fla...,5.0


In [14]:
X = review_data['full_review_text']
y = review_data['star_rating']

In [15]:
X = pd.Series([str(text) for text in X])

In [16]:
review_data.isnull().sum()

full_review_text    44
star_rating          0
dtype: int64

In [17]:
review_data.dropna()

,full_review_text,star_rating
0,decide eat aware go take 2 hour begin end try ...,3.0
1,ve take lot spin class year nothing compare cl...,5.0
2,family diner buffet eclectic assortment large ...,3.0
3,wow yummy different delicious favorite lamb cu...,5.0
4,cute interior owner give u tour upcoming patio...,4.0
...,...,...
6990275,late addition service iccu apple pay iccu debi...,5.0
6990276,spot offer great affordable east weekend paddl...,5.0
6990277,home depot need get lot demential lumber seem ...,4.0
6990278,m feel like ignore caloriecounting indulge fla...,5.0


In [ ]:
review_data['full_review_text'].replace('', np.nan, inplace=True)
review_data.dropna(inplace = True)

In [18]:
if not os.path.exists('../pickle_files'):
    os.mkdir('../pickle_files')

In [ ]:
if not os.path.exists('../saved_embeddings'):
    os.mkdir('../saved_embeddings')

In [ ]:
UNIQUE_WORDS = set(' '.join(X).split())

In [ ]:
VOCAB_SIZE = len(UNIQUE_WORDS)
EMBEDDING_DIM = 100
MAX_TEXT_LEN = 200
TRUNC_TYPE = 'post'
PADDING_TYPE = 'post'
OOV_TOKEN = "<OOV>"
BATCH_SIZE = 64
EPOCHS = 15

### Word2Vec

In [23]:
count_model = CountVectorizer()

In [ ]:
if not os.path.exists("../saved_embeddings/w2v_model.model"):
    print("Creating Embedding from scratch.")
    w2v_model = Word2Vec(sentences=[nltk.word_tokenize(text) for text in X], vector_size=100, window=5, min_count=1, workers=4)
    w2v_model.save("../saved_embeddings/w2v_model.model")
else:
    print("Embedding found.")
    w2v_model = Word2Vec.load("../saved_embeddings/w2v_model.model")

In [ ]:
w2v_model = KeyedVectors.load_word2vec_format("../saved_embeddings/w2v_model.model", binary=True)

In [ ]:
word_index_gensim = {}

for i, word in enumerate(w2v_model.vocab):
    word_index_gensim[word] = i

In [ ]:
vocab_size = len(word_index_gensim)
embedding_dim_gensim = w2v_model.vector_size

embedding_matrix_gensim = np.zeros((vocab_size, embedding_dim_gensim))

for word, i in word_index_gensim.items():
    if word in w2v_model.vocab:
        embedding_matrix_gensim[i] = w2v_model[word]

EMBEDDING_LAYER_GENSIM = Embedding(vocab_size,
                            embedding_dim_gensim,
                            weights=[embedding_matrix_gensim],
                            trainable=False)

### GloVe

In [ ]:
if not os.path.exists('../glove_files'):
    os.mkdir('../glove_files')

In [ ]:
if not os.path.exists('../glove_files/glove.6B.100d.txt'):
    print("Downloading GloVe File...")
    r = requests.get(GLOVE_URL)
    with open("../glove_files/glove.6B.100d.txt", 'wb') as fd:
        for data in r.iter_content():
            #save file in the current directory of the notebook
            fd.write(data)

In [ ]:
TRAIN_PERCENT = 0.8
VALID_TEST_PERCENT = 0.1
TRAIN_SIZE = int(ACTUAL_DATA_SHAPE * TRAIN_PERCENT)
VALID_TEST_SIZE = int(ACTUAL_DATA_SHAPE * VALID_TEST_PERCENT)
TOTAL_TEST_SIZE = TRAIN_SIZE + VALID_TEST_SIZE
train_set = X[:TRAIN_SIZE]
train_labels = y[:TRAIN_SIZE]
validation_set = X[TRAIN_SIZE:TOTAL_TEST_SIZE]
validation_labels = y[TRAIN_SIZE:TOTAL_TEST_SIZE]
test_set = X[TOTAL_TEST_SIZE:]
test_labels = y[TOTAL_TEST_SIZE:]

In [ ]:
READY_DATASET_PATH = "../larger_dataset/ready_for_models/"

In [ ]:
train_set_padded = pickle.load(open(READY_DATASET_PATH + "train_set_padded.pickle", "rb"))
train_labels = pickle.load(open(READY_DATASET_PATH + "train_labels.pickle", "rb"))
valid_set_padded = pickle.load(open(READY_DATASET_PATH + "valid_set_padded.pickle", "rb"))
validation_labels = pickle.load(open(READY_DATASET_PATH + "validation_labels.pickle", "rb"))
test_set_padded = pickle.load(open(READY_DATASET_PATH + "test_set_padded.pickle", "rb"))
test_labels = pickle.load(open(READY_DATASET_PATH + "test_labels.pickle", "rb"))
EMBEDDING_LAYER_GLOVE = pickle.load(open(READY_DATASET_PATH + "EMBEDDING_LAYER.pickle", "rb"))

### Class Weights

In [ ]:
class_weights = compute_class_weight('balanced', classes=[0,1,2,3,4], y=y)

### RNN Model With Word2Vec

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
rnn_model_w2v = tf.keras.models.Sequential([
    EMBEDDING_LAYER_GENSIM,
    tf.keras.layers.SimpleRNN(60),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dense(5, activation = "softmax")
])

2023-05-03 15:23:40.175617: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-03 15:23:40.183282: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-03 15:23:40.183681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
rnn_model_w2v.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          100208800 
                                                                 
 simple_rnn (SimpleRNN)      (None, 60)                9660      
                                                                 
 dense (Dense)               (None, 512)               31232     
                                                                 
 dense_1 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 100,252,257
Trainable params: 43,457
Non-trainable params: 100,208,800
_________________________________________________________________


In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [ ]:
rnn_model_w2v.compile(optimizer=Adam(), loss = SparseCategoricalCrossentropy(), metrics=['accuracy',custom_f1_score])
rnn_model_w2v.fit(train_set_padded, train_labels,validation_data = (valid_set_padded,validation_labels),\
              batch_size=BATCH_SIZE, epochs=EPOCHS,class_weight=dict(enumerate(class_weights)),\
              callbacks=[tensorboard_callback,EarlyStopping(patience=3),ReduceLROnPlateau(factor=0.1, patience=1)])

Epoch 1/15


2023-05-03 15:23:41.291114: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4473779200 exceeds 10% of free system memory.
2023-05-03 15:23:44.387572: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-03 15:23:44.426304: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0xfed046a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-03 15:23:44.426330: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-05-03 15:23:44.429885: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-03 15:23:44.552948: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-03 15:23:44.655573: I

 7346/87379 [=>............................] - ETA: 1:33:07 - loss: 1.6083 - accuracy: 0.2714 - get_f1: 7.8349

### RNN Model with GloVe

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
rnn_model_glove = tf.keras.models.Sequential([
    EMBEDDING_LAYER_GLOVE,
    tf.keras.layers.SimpleRNN(60),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dense(5, activation = "softmax")
])

2023-05-03 15:23:40.175617: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-03 15:23:40.183282: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-03 15:23:40.183681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
rnn_model_glove.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          100208800 
                                                                 
 simple_rnn (SimpleRNN)      (None, 60)                9660      
                                                                 
 dense (Dense)               (None, 512)               31232     
                                                                 
 dense_1 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 100,252,257
Trainable params: 43,457
Non-trainable params: 100,208,800
_________________________________________________________________


In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [ ]:
rnn_model_glove.compile(optimizer=Adam(), loss = SparseCategoricalCrossentropy(), metrics=['accuracy',custom_f1_score])
rnn_model_glove.fit(train_set_padded, train_labels,validation_data = (valid_set_padded,validation_labels),\
              batch_size=BATCH_SIZE, epochs=EPOCHS,class_weight=dict(enumerate(class_weights)),\
              callbacks=[tensorboard_callback,EarlyStopping(patience=3),ReduceLROnPlateau(factor=0.1, patience=1)])

Epoch 1/15


2023-05-03 15:23:41.291114: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4473779200 exceeds 10% of free system memory.
2023-05-03 15:23:44.387572: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-03 15:23:44.426304: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0xfed046a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-03 15:23:44.426330: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-05-03 15:23:44.429885: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-03 15:23:44.552948: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-03 15:23:44.655573: I

 7346/87379 [=>............................] - ETA: 1:33:07 - loss: 1.6083 - accuracy: 0.2714 - get_f1: 7.8349